In [ ]:
!pip install --upgrade wandb

# Importing Libraries

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import wandb
import os
from typing import Any, Dict, List
import copy
import random
import wandb
import numpy as np
import matplotlib.pyplot as plt

#Libraries for mobolity
from collections import Counter
import random
import numpy as np
import logging
import clustering
import clients
import numpy
from scipy.spatial.distance import cdist
from mobility import gauss_markov, reference_point_group, \
    tvc, truncated_levy_walk, random_direction, random_waypoint, random_walk
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import Leach
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

#Clustering Library
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import kmean_clustering

#custom library
import custom_model
import server



In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#os.environ["WANDB_API_KEY"] = "183c1a6a36cbdf0405f5baacb72690845ecc8573"

# Importing dataset
## validating dataset structure

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='cifar_data', train=True,download=True,transform=transform)
testset = torchvision.datasets.CIFAR10(root='cifar_data', train=False, download=True,transform=transform)

print(trainset)
print(testset)

trainloader = torch.utils.data.DataLoader(trainset, 
                                           batch_size=500,
                                           shuffle=True)

print(str("\ntrainset datatype: ")+str((type(trainset))))
print(str("testset datatype: ")+str((type(testset))))


print(str("\nLabel in cifar10: ")+str(np.unique([trainset[i][1] for i in range(len(trainset))])))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(classes)


def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join(f'{classes[labels[j]]:1s}' for j in range(16)))

# # This is for iid data
# trainloader = torch.utils.data.DataLoader(trainset, 
#                                           batch_size=500,
#                                           shuffle=True)
testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=100,
                                        shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: cifar_data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: cifar_data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

trainset datatype: <class 'torchvision.datasets.cifar.CIFAR10'>
testset datatype: <class 'torchvision.datasets.cifar.CIFAR10'>

Label in cifar10: [0 1 2 3 4 5 6 7 8 9]
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Initialising parameters


In [6]:
n_client = 30
n_leader = int(n_client * 0.1)
chosen_prob = 0.8
local_batch_size = 32
local_epochs = 5

epochs = 5
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD
optimizer_conf = dict(
    lr=0.001,
    momentum=0.9
)



model = custom_model.Net().to(device)
print(str("Model structure: \n")+str(model))

server_list: List[server.Server] = []

leaderStep = int(n_client/n_leader)

Model structure: 
Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Mobility
## Assigning coordinates to nodes & creating clusters using Leach protocol

In [7]:
classes_pair = [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]

data_label = np.array(trainset.targets)

nr_nodes = int(n_client)

logger = logging.getLogger("vanet")

step = 0
np.random.seed(0xffff)

# simulation area (units)
MAX_X, MAX_Y = 100, 100

# max and min velocity
MIN_V, MAX_V = 0.1, 1.

# max waiting time
MAX_WT = 100.

# number of steps to ignore before start plotting
STEPS_TO_IGNORE = 10000

# set this to true if you want to calculate node contacts
CALCULATE_CONTACTS = False

# if calculating contacts, this is the range to be used
# (if a distance(a,b) < RANGE, then there is a contact betwen a and b)
RANGE = 1.

#rw = random_walk(nr_nodes, dimensions=(MAX_X, MAX_Y))
#rw = gauss_markov(nr_nodes, dimensions=(MAX_X, MAX_Y))
#rw = truncated_levy_walk(nr_nodes, dimensions=(MAX_X, MAX_Y))
rw = reference_point_group(nr_nodes, dimensions=(MAX_X, MAX_Y),velocity=(5.0, 10.0))

cluster = clustering.Cluster(nr_nodes,
                             step,MAX_X, 
                             MAX_Y,MIN_V, 
                             MAX_V,MAX_WT,
                             STEPS_TO_IGNORE,
                             CALCULATE_CONTACTS,
                             RANGE,rw)

coordinate = cluster.coord(rw)
print(str("Coordinates generated: \n") + str(coordinate))

leach = Leach.Leach(coordinate)

node = leach.assign_energy(coordinate)

array_header = leach.define_header(node,coordinate)

print(str("\nCluster Heads generated: \n")+str(array_header))

headers = np.array(array_header)

header_coord_index = []
for i in range(len(coordinate)):
    for j in range(len(headers)):
        if (coordinate[i][0]) == (headers[j][0]):
            header_coord_index.append(i)

print(str("\nheader_coord: \n")+str(header_coord_index))
distance = []

for i in range(len(array_header)):
    for j in range(len(coordinate)):
        dist = leach.calculate_distance(coordinate[j][0],
                                  array_header[i][0],
                                  coordinate[j][1],
                                  array_header[i][1])
        distance.append(dist)
        
newdistance = np.array_split(distance, len(array_header))
#print(newdistance)

clusters = leach.assign_cluster(newdistance)
            
print(str("\nclusters generated: \n")+str(clusters))

        
leach.plot_cluster(clusters,coordinate)

coordinate_cluster = [[] for _ in range(len(array_header))]

for i in range(len(array_header)):
    for j in range(len(clusters)):
        if clusters[j] == i:
            coordinate_cluster[i].append(coordinate[j])

#print(coordinate_cluster[0])

client_list = []

for i in range(len(coordinate_cluster)):
    client_list.append(len(coordinate_cluster[i]))
    
#print(client_list)

cluster_number = len(array_header)

max_value = numpy.max(clusters)

cluster_coords = leach.initiaiseClusterCoordinates(len(array_header),clusters)

cluster_coords = leach.fillingNoneValue(cluster_coords,cluster_number,n_client)


Coordinates generated: 
[[18.39839891 26.79646631]
 [95.03824448 61.86341421]
 [90.38934082 26.38757339]
 [53.34519328 56.74059427]
 [39.41817051 10.90634014]
 [28.42918166 56.92422588]
 [80.42768    61.58491742]
 [32.02827993 24.42435152]
 [85.68114019 41.30603091]
 [86.33075913 25.14261621]
 [76.18378956 65.86985846]
 [47.72839356 66.39393991]
 [20.25853068 91.03235363]
 [18.43027994 95.53617346]
 [35.96631085 12.61518217]
 [61.78414449  2.45425339]
 [76.19556261 69.10506403]
 [62.37692549  1.44829415]
 [38.21671773 20.38841057]
 [63.93959551 86.84645104]
 [47.15005807  2.66312755]
 [10.8449529  57.94976435]
 [73.43714486  5.15827358]
 [47.86458779 90.63239674]
 [52.63976174 89.99001038]
 [13.13148564 51.57567912]
 [56.58021295 76.97139824]
 [69.62869179 99.03848919]
 [49.14497882 78.8885162 ]
 [18.26780863 32.33673877]]

Cluster Heads generated: 
[array([95.03824448, 61.86341421]), array([76.18378956, 65.86985846]), array([62.37692549,  1.44829415]), array([38.21671773, 20.38841057]

In [8]:
servers = []

for i in range(0,len(array_header)):
    server_list = server.Server(
        model=model,
        loss=criteria,
        optimizer=optimizer,
        n_client=client_list[i],
        #n_leader=n_leader,
        chosen_prob=chosen_prob,
        optimizer_conf=optimizer_conf,
        local_batch_size=local_batch_size,
        local_epochs=local_epochs
    )
    servers.append(server_list)
    
def initialiseClientCoordinates(best_k,coordinate,cluster_coords):
    for i in range(int(best_k)):
        for j in range(len(coordinate)):
            for k in range(len(cluster_coords[i])):
                #print(str(cluster_coords[i][j])+ str(' coord: ') + str(coordinate[k]))
                if (cluster_coords[i][j] == k):
                    servers[i].client_pool[j].setXCoordinate(coordinate[k][0])
                    servers[i].client_pool[j].setYCoordinate(coordinate[k][1])

        
initialiseClientCoordinates(len(array_header),coordinate,cluster_coords)

def showClient():
    for i in range(int(best_k)):
        for j in range(client_list[i]):
            print(str('for server: ') + str(i) + str(' client ') + str(j) 
                  + str(' coordinates ') + str(servers[i].client_pool[j].actual_Xcoord) + str(', ') 
                  + str(servers[i].client_pool[j].actual_Ycoord))
            
#showClient()

In [9]:

# # This is for iid data
# for batch_idx, (batch_feature, batch_label) in enumerate(trainloader):
#     server.client_pool[batch_idx].setData(list(zip(batch_feature, batch_label)))

# This is for non-iid data                   
chosen_counter = Counter()
for i in range(len(servers)):
    for _, client in tqdm(enumerate(servers[i].client_pool)):
        # sample until we have a pair of class with insufficient client owning
        class_pair = random.choice(classes_pair)
        while chosen_counter[class_pair] == 100:
            class_pair = random.choice(classes_pair)

        chosen_counter[class_pair] += 1
        first_class, second_class = class_pair
        first_class_sample_idx = list(np.where(data_label == first_class)[0])
        second_class_sample_idx = list(np.where(data_label == second_class)[0])
        #first_class_sample_idx = list(first_class[0])
        #second_class_sample_idx = list(scond_class[0])

        client_first_class_sample_idx = random.sample(first_class_sample_idx, k=500)
        client_second_class_sample_idx = random.sample(second_class_sample_idx, k=500)
        client_data = []
    
        for i in range(500):
            client_data.append(trainset[client_first_class_sample_idx[i]])
            client_data.append(trainset[client_second_class_sample_idx[i]])

        client.setData(client_data)
        



2it [00:00,  6.03it/s]
9it [00:01,  6.28it/s]
0it [00:00, ?it/s]
5it [00:00,  5.60it/s]
14it [00:03,  4.34it/s]


# Dynamic nodes clustering using Leach


In [10]:
#%matplotlib
#looping process

coordinate_array = []
best_ks = []
cluster_array = []
kmeans_array = []
cluster_coords_array = []

LABEL_COLOR_MAP = {0 : 'r',
                   1 : 'g',
                   2 : 'b',
                   3 : 'y',
                   4 : 'b'
                   }

#label_color = [LABEL_COLOR_MAP[l] for l in kmeans.labels_]

%matplotlib  
for i in range(epochs):
    coordinate = cluster.coord(rw)
    leach = Leach.Leach(coordinate)
    new_coord = leach.speedRate(coordinate)
    new_coord = np.array(new_coord)
    print(new_coord)  
    node = leach.assign_energy(new_coord)
   
    array_header = leach.define_header(node,new_coord)
    distance = []
    
    for j in range(len(array_header)):
        for k in range(len(new_coord)):
            dist = leach.calculate_distance(new_coord[k][0],
                                      array_header[j][0],
                                      new_coord[k][1],
                                      array_header[j][1])
            distance.append(dist)      
    newdistance = np.array_split(distance, len(array_header))
    clusters = leach.assign_cluster(newdistance)
            
    print(clusters)  
         
    leach.plot_cluster(clusters,new_coord)

Using matplotlib backend: Qt5Agg
[[ 15.42016394  24.5889661 ]
 [ 92.04006623  59.92374819]
 [ 87.71253018  25.12373056]
 [ 52.08954697  55.40837986]
 [ 37.96633874   8.06997205]
 [ 25.7659834   55.67266965]
 [ 78.1911557   60.556543  ]
 [ 30.89005269  22.93164603]
 [ 82.81116382  38.81293751]
 [ 83.39121971  23.48505698]
 [ 74.96751658  64.49096391]
 [ 44.89258719  63.84491559]
 [ 18.36837627  88.03840497]
 [ 16.10568532  92.59032022]
 [ 34.92539973  10.89828731]
 [183.08915312   5.39360755]
 [ 74.25688639  66.1069461 ]
 [184.9175541    6.37013474]
 [ 36.7856348   19.21080547]
 [ 60.98053555  84.56324793]
 [ 44.9964143    1.65125385]
 [  8.13873469  55.24177011]
 [ 70.58242515   3.67736338]
 [ 45.59829268  89.59628829]
 [ 49.96938177  87.24799239]
 [ 10.14893509  49.38968333]
 [ 55.29852647  75.66711779]
 [ 67.59667683  97.96202342]
 [ 47.14372197  77.88851541]
 [ 16.28739097  31.33654702]]
[4, 0, 4, 3, 4, 3, 1, 4, 0, 4, 1, 3, 3, 3, 4, 2, 1, 2, 4, 1, 4, 4, 4, 1, 1, 4, 1, 1, 1, 4]
Using

In [20]:
#wandb.init(project="fl", name="CNN_CIFAR_10_noniid")
#print((servers[0].client_pool[0].data))
aggregates = []

    
for i in range(epochs):
    for j in range(len(servers)):
        servers[j].aggregate()
        servers[j].broadcast()
        total_correct = 0
        #with torch.no_grad():
        for _, (test_feature, test_label) in enumerate(testloader):
            test_feature = test_feature.to(device)
            test_label = test_label.to(device)
            y_pred = servers[j].model(test_feature)
            y_pred_decode = torch.argmax(y_pred, dim=1)

            total_correct += y_pred_decode.eq(test_label).sum().item()
            #print(str('total_correct ')+str(total_correct))
        test_acc = total_correct / 10000

        print("Overall acc: {}, overall_loss: {}".format(servers[j].avg_acc, servers[j].avg_loss))
        #wandb.log({"acc": servers[j].avg_acc, "loss": servers[j].avg_loss})


Original Secret: 1923275859
Shares: (30261, 3294432030), (88716, 5943086535)
Reconstructed secret: 3294432030
Client 0: Acc 0.5234375, Loss: 0.7233080882579088
Overall acc: 0.5234375, overall_loss: 0.7233080882579088
Original Secret: 2645221515
Shares: (94849, 10469315525), (38849, 5849875525), (15846, 3952358055), (97406, 10680242455), (43608, 6242445435), (94567, 10446053345), (93218, 10334774335), (65641, 8059947605), (55327, 7209145745)
Reconstructed secret: 2645221515
Client 7: Acc 0.5224609375, Loss: 0.6914281006902456
Client 4: Acc 0.50390625, Loss: 0.7342945728451014


KeyboardInterrupt: 